In [79]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import math

df = pd.read_csv("./ufc.csv")
df = pd.DataFrame(df)
df

,Location,Fighter 1,Fighter 2,Fighter_1_KD,Fighter_2_KD,Fighter_1_STR,Fighter_2_STR,Fighter_1_TD,Fighter_2_TD,Fighter_1_SUB,Fighter_2_SUB,Weight_Class,Method,Round,Time,Event Name,Date,Winner
0,"Austin, Texas, USA",Arman Tsarukyan,Beneil Dariush,1.0,0.0,8.0,2.0,0.0,0.0,0.0,0.0,Lightweight,KO/TKO Punch,1.0,1:04,UFC Fight Night,2-Dec-23,Arman Tsarukyan
1,"Austin, Texas, USA",Jalin Turner,Bobby Green,1.0,0.0,33.0,15.0,0.0,0.0,0.0,0.0,Lightweight,KO/TKO Punches,1.0,2:49,UFC Fight Night,2-Dec-23,Jalin Turner
2,"Austin, Texas, USA",Deiveson Figueiredo,Rob Font,0.0,0.0,45.0,46.0,4.0,0.0,0.0,0.0,Bantamweight,U-DEC,3.0,5:00,UFC Fight Night,2-Dec-23,Deiveson Figueiredo
3,"Austin, Texas, USA",Sean Brady,Kelvin Gastelum,0.0,0.0,14.0,18.0,5.0,0.0,3.0,0.0,Welterweight,SUB Kimura,3.0,1:43,UFC Fight Night,2-Dec-23,Sean Brady
4,"Austin, Texas, USA",Joaquim Silva,Clay Guida,0.0,0.0,46.0,43.0,2.0,2.0,2.0,0.0,Lightweight,U-DEC,3.0,5:00,UFC Fight Night,2-Dec-23,Joaquim Silva
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7412,"Denver, Colorado, USA",Orlando Wiet,Sean Daugherty,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,Open Weight,SUB Guillotine Choke,1.0,0:20,UFC 2,11-Mar-94,Patrick Smith
7413,"Denver, Colorado, USA",Frank Hamaker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Scott Morris
7414,"Denver, Colorado, USA",Johnny Rhodes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7415,"Denver, Colorado, USA",Patrick Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
df_new = df.dropna()

def time_converter(time):
    splitted = time.split(":")

    first = int(splitted[0])
    second = int(splitted[1])

    res = first*60 + second

    return res

def round_converter(rnd):
    return int(rnd) * 300

df_new["Time in seconds"] = df_new["Time"].apply(time_converter)

df_new["Time in seconds"] += df_new["Round"].apply(round_converter)

df_new


/var/folders/lh/rd5kzgf52pv46_540yq0yf380000gn/T/ipykernel_52240/2729609065.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["Time in seconds"] = df_new["Time"].apply(time_converter)
/var/folders/lh/rd5kzgf52pv46_540yq0yf380000gn/T/ipykernel_52240/2729609065.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["Time in seconds"] += df_new["Round"].apply(round_converter)


,Location,Fighter 1,Fighter 2,Fighter_1_KD,Fighter_2_KD,Fighter_1_STR,Fighter_2_STR,Fighter_1_TD,Fighter_2_TD,Fighter_1_SUB,Fighter_2_SUB,Weight_Class,Method,Round,Time,Event Name,Date,Winner,Time in seconds
0,"Austin, Texas, USA",Arman Tsarukyan,Beneil Dariush,1.0,0.0,8.0,2.0,0.0,0.0,0.0,0.0,Lightweight,KO/TKO Punch,1.0,1:04,UFC Fight Night,2-Dec-23,Arman Tsarukyan,364
1,"Austin, Texas, USA",Jalin Turner,Bobby Green,1.0,0.0,33.0,15.0,0.0,0.0,0.0,0.0,Lightweight,KO/TKO Punches,1.0,2:49,UFC Fight Night,2-Dec-23,Jalin Turner,469
2,"Austin, Texas, USA",Deiveson Figueiredo,Rob Font,0.0,0.0,45.0,46.0,4.0,0.0,0.0,0.0,Bantamweight,U-DEC,3.0,5:00,UFC Fight Night,2-Dec-23,Deiveson Figueiredo,1200
3,"Austin, Texas, USA",Sean Brady,Kelvin Gastelum,0.0,0.0,14.0,18.0,5.0,0.0,3.0,0.0,Welterweight,SUB Kimura,3.0,1:43,UFC Fight Night,2-Dec-23,Sean Brady,1003
4,"Austin, Texas, USA",Joaquim Silva,Clay Guida,0.0,0.0,46.0,43.0,2.0,2.0,2.0,0.0,Lightweight,U-DEC,3.0,5:00,UFC Fight Night,2-Dec-23,Joaquim Silva,1200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7408,"Denver, Colorado, USA",Patrick Smith,Robert Lucarelli,0.0,0.0,8.0,2.0,0.0,1.0,0.0,1.0,Open Weight,KO/TKO,1.0,2:50,UFC 2,11-Mar-94,Remco Pardoel,470
7409,"Denver, Colorado, USA",Royce Gracie,Thaddeus Luster,0.0,0.0,2.0,0.0,1.0,0.0,3.0,0.0,Open Weight,SUB Keylock,1.0,4:52,UFC 2,11-Mar-94,Orlando Wiet,592
7410,"Denver, Colorado, USA",Jason DeLucia,David Levicki,0.0,0.0,11.0,4.0,1.0,0.0,0.0,0.0,Open Weight,KO/TKO Punches,1.0,12:13,UFC 2,11-Mar-94,Frank Hamaker,1033
7411,"Denver, Colorado, USA",Remco Pardoel,Ray Wizard,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,Open Weight,SUB Guillotine Choke,1.0,0:58,UFC 2,11-Mar-94,Johnny Rhodes,358


In [81]:
total_kd = (df["Fighter_1_KD"] + df["Fighter_2_KD"]).mean()
total_str= (df["Fighter_1_STR"] + df["Fighter_2_STR"]).mean()
total_td= (df["Fighter_1_TD"] + df["Fighter_2_TD"]).mean()
total_sub= (df["Fighter_1_SUB"] + df["Fighter_1_SUB"]).mean()

print("Average knockdowns per fight: ", total_kd)
print("Average significant strikes per fight: ", total_str)
print("Average takedown attemps per fight: ", total_td)
print("Average submissions attemps per fight: ", total_sub)

Average knockdowns per fight:  0.43059490084985835
Average significant strikes per fight:  71.67637933360312
Average takedown attemps per fight:  2.1181707810602997
Average submissions attemps per fight:  1.0640766221502767


In [82]:
event_names = df_new["Event Name"].unique() # useless
methods = df_new["Method"].unique()

methods

array(['KO/TKO Punch', 'KO/TKO Punches', 'U-DEC', 'SUB Kimura',
       'SUB Rear Naked Choke', 'KO/TKO Slam', 'S-DEC', 'SUB Neck Crank',
       'SUB Guillotine Choke', 'Overturned', 'KO/TKO Elbows',
       'KO/TKO Kick', 'SUB Armbar', 'M-DEC', 'KO/TKO Knees', 'CNC',
       'SUB Arm Triangle', 'KO/TKO Kicks', 'KO/TKO Knee', 'KO/TKO', 'DQ',
       'SUB Anaconda Choke', 'SUB Ezekiel Choke', 'SUB Triangle Choke',
       'SUB Twister', 'SUB Triangle Armbar', "SUB D'Arce Choke",
       'KO/TKO Spinning Back Elbow', 'Overturned Guillotine Choke',
       'KO/TKO Elbow', 'SUB Other - Lock', 'SUB Kneebar',
       'SUB Inverted Triangle', 'KO/TKO Flying Knee',
       'KO/TKO Spinning Back Kick', 'SUB Von Flue Choke',
       'KO/TKO Spinning Back Fist', 'SUB Keylock', 'SUB Scarf Hold',
       'SUB Straight Armbar', 'SUB Heel Hook', 'Overturned Punches',
       'SUB Ankle Lock', 'SUB Forearm Choke', 'SUB Injury',
       'Overturned Punch', 'Overturned Rear Naked Choke',
       'Overturned Knee', 'O

Тут можна теж багато різного побачити. Я б казав забрати все що не містить у собі KO/TKO, DEC, SUB, CNC (зупинка від команди типу білий рушник кинули). Overtuned, Other, DQ - не пригодяться нам (перше вроді про неправильне рішення суддів напевно, а про друге не знаю)

In [83]:
def methods_destroyer(df: pd.DataFrame):
    filtered = df[~df_new["Method"].str.contains("Overturned|DQ|Other")]
    return filtered


df_fil = methods_destroyer(df_new)

df_fil["Method"].unique()

array(['KO/TKO Punch', 'KO/TKO Punches', 'U-DEC', 'SUB Kimura',
       'SUB Rear Naked Choke', 'KO/TKO Slam', 'S-DEC', 'SUB Neck Crank',
       'SUB Guillotine Choke', 'KO/TKO Elbows', 'KO/TKO Kick',
       'SUB Armbar', 'M-DEC', 'KO/TKO Knees', 'CNC', 'SUB Arm Triangle',
       'KO/TKO Kicks', 'KO/TKO Knee', 'KO/TKO', 'SUB Anaconda Choke',
       'SUB Ezekiel Choke', 'SUB Triangle Choke', 'SUB Twister',
       'SUB Triangle Armbar', "SUB D'Arce Choke",
       'KO/TKO Spinning Back Elbow', 'KO/TKO Elbow', 'SUB Kneebar',
       'SUB Inverted Triangle', 'KO/TKO Flying Knee',
       'KO/TKO Spinning Back Kick', 'SUB Von Flue Choke',
       'KO/TKO Spinning Back Fist', 'SUB Keylock', 'SUB Scarf Hold',
       'SUB Straight Armbar', 'SUB Heel Hook', 'SUB Ankle Lock',
       'SUB Forearm Choke', 'SUB Injury', 'SUB Peruvian Necktie',
       'SUB Schultz Front Headlock', 'SUB Bulldog Choke',
       'SUB Suloev Stretch', 'SUB Omoplata', 'SUB Calf Slicer',
       'SUB North-South Choke', 'SUB Pac

In [84]:
df_new["Weight_Class"].unique()

array(['Lightweight', 'Bantamweight', 'Welterweight', 'Middleweight',
       "Women's Bantamweight", 'Light Heavyweight', "Women's Flyweight",
       "Women's Strawweight", 'Featherweight', 'Heavyweight', 'Flyweight',
       'Catch Weight', "Women's Featherweight", 'Super Heavyweight',
       'Open Weight'], dtype=object)

Open weight та catch weight - можна забрати бо це не стандартна вага

In [85]:
def weight_destroyer(df: pd.DataFrame):
    filtered = df[~df_new["Weight_Class"].str.contains("Open|Catch")]
    return filtered

df_fil = weight_destroyer(df_fil)

df_fil["Weight_Class"].unique()

/var/folders/lh/rd5kzgf52pv46_540yq0yf380000gn/T/ipykernel_52240/2663720929.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = df[~df_new["Weight_Class"].str.contains("Open|Catch")]


array(['Lightweight', 'Bantamweight', 'Welterweight', 'Middleweight',
       "Women's Bantamweight", 'Light Heavyweight', "Women's Flyweight",
       "Women's Strawweight", 'Featherweight', 'Heavyweight', 'Flyweight',
       "Women's Featherweight", 'Super Heavyweight'], dtype=object)

Можна викинути колонки про Date і Event

In [86]:
def de_remover(df: pd.DataFrame):
    if df.columns.str.contains("Date|Event Name").any():
        return df.drop(["Date", "Event Name"], axis=1)
    

df_fil = de_remover(df_fil)


In [87]:
def replace_method(method: str):
    if "DEC" in method:
        return "DEC"
    elif "SUB" in method:
        return "SUB"
    elif "KO" in method:
        return "KO"
    elif "CNC" in method:
        return "KO"

df_fil["Method"] = df_fil["Method"].apply(replace_method)


df_fil["Method"].unique()


array(['KO', 'DEC', 'SUB'], dtype=object)

In [88]:
fighters = list(set(df_fil["Fighter 1"].unique()).union(set(df_fil["Fighter 2"].unique())))
fighters_df = pd.DataFrame({"Fighter_name": [],
                            "Average_significant_strikes": [],
                            "Average_takedowns": [],
                            "Average_submission_attempts": [],
                            "Wins": [],
                            "Loss": []})


for ele in fighters:
    win_df = df_fil.loc[df_fil["Fighter 1"] == ele]
    lose_df = df_fil.loc[df_fil["Fighter 2"] == ele]

    wins, losses = int(len(win_df.index)), int(len(lose_df.index))

    amount_of_mentions = wins + losses

    avg_strk = round((win_df["Fighter_1_STR"].sum() + lose_df["Fighter_2_STR"].sum())/amount_of_mentions, 2)
    avg_tkd = round((win_df["Fighter_1_TD"].sum() + lose_df["Fighter_2_TD"].sum())/amount_of_mentions, 2)
    avg_subs = round((win_df["Fighter_1_SUB"].sum() + lose_df["Fighter_2_SUB"].sum())/amount_of_mentions, 2)

    data = {
        "Fighter_name": ele,
        "Average_significant_strikes": [avg_strk],
        "Average_takedowns": [avg_tkd],
        "Average_submission_attempts": [avg_subs],
        "Wins": [wins],
        "Loss": [losses]
    }

    fighters_df = pd.concat([fighters_df, pd.DataFrame(data)], ignore_index = True)



fighters_df

,Fighter_name,Average_significant_strikes,Average_takedowns,Average_submission_attempts,Wins,Loss
0,Phil Johns,2.00,0.00,0.00,0.0,1.0
1,Matt Lindland,24.18,2.00,0.82,8.0,3.0
2,Ben Saunders,25.58,0.11,0.68,9.0,10.0
3,Andre Gusmao,25.50,0.00,0.00,0.0,2.0
4,Tom Breese,23.50,0.00,0.50,5.0,3.0
...,...,...,...,...,...,...
2343,Bruno Santos,17.67,3.00,0.33,1.0,2.0
2344,Katsunori Kikuno,15.80,0.80,0.20,2.0,3.0
2345,Gilbert Smith,14.00,3.00,1.00,0.0,1.0
2346,Cheyanne Vlismas,52.25,0.25,0.25,2.0,2.0


In [89]:
fighters_df.loc[fighters_df["Average_takedowns"] == fighters_df["Average_takedowns"].max()]

,Fighter_name,Average_significant_strikes,Average_takedowns,Average_submission_attempts,Wins,Loss
36,Myktybek Orolbai,8.0,7.0,1.0,1.0,0.0


In [90]:
# fighters_df = fighters_df.drop(["Fighter_name"], axis = 1)

model = smf.ols(f"Wins ~ {' + '.join(fighters_df.drop(['Wins', 'Loss'], axis=1).columns.to_list())}", data=fighters_df).fit()

print(model.summary())



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


                            OLS Regression Results                            
Dep. Variable:                   Wins   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 08 Apr 2024   Prob (F-statistic):                nan
Time:                        10:45:54   Log-Likelihood:                 65637.
No. Observations:                2348   AIC:                        -1.266e+05
Df Residuals:                       0   BIC:                        -1.131e+05
Df Model:                        2347                                         
Covariance Type:            nonrobust                                         
                                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------

# OLS

In [91]:
df_fil.columns

Index(['Location', 'Fighter 1', 'Fighter 2', 'Fighter_1_KD', 'Fighter_2_KD',
       'Fighter_1_STR', 'Fighter_2_STR', 'Fighter_1_TD', 'Fighter_2_TD',
       'Fighter_1_SUB', 'Fighter_2_SUB', 'Weight_Class', 'Method', 'Round',
       'Time', 'Winner', 'Time in seconds'],
      dtype='object')

In [92]:
df_fil["Weight_Class"].unique()

array(['Lightweight', 'Bantamweight', 'Welterweight', 'Middleweight',
       "Women's Bantamweight", 'Light Heavyweight', "Women's Flyweight",
       "Women's Strawweight", 'Featherweight', 'Heavyweight', 'Flyweight',
       "Women's Featherweight", 'Super Heavyweight'], dtype=object)

In [93]:
df_fil["Gender"] = np.where(df_fil["Weight_Class"].str.contains("Women"), 1, 0)

In [94]:
df_man = df_fil.loc[df_fil["Gender"] == 0].drop(["Gender"], axis=1)
df_woman = df_fil.loc[df_fil["Gender"] != 0].drop(["Gender"], axis=1)

In [95]:
def weight_breaker(df: pd.DataFrame):
    for ele in df["Weight_Class"].unique():
        df[ele] = 1

    for ele in df["Weight_Class"].unique():
        df[ele] = np.where(df["Weight_Class"].str.contains(ele), 1, 0)

    # if df.columns.str.contains("Weight_Class"):
    #     df.drop(["Weight_Class"], axis=1)

    return df

df_man_clear = weight_breaker(df_man)
df_woman_clear = weight_breaker(df_woman)


In [96]:
df_man_fixed = df_man_clear.loc[(df_man_clear["Winner"] == df_man_clear["Fighter 1"]) | (df_man_clear["Winner"] == df_man_clear["Fighter 2"])].reset_index()
df_woman_fixed = df_woman_clear.loc[(df_woman_clear["Winner"] == df_woman_clear["Fighter 1"]) | (df_woman_clear["Winner"] == df_woman_clear["Fighter 2"])].reset_index()

In [97]:
def history_winrate(df: pd.DataFrame):
    nrows = df.size//df.columns.size
    winrate = []

    for i in range(nrows):
        current = df.iloc[i,]

        for j in range(1,3):
            current_fighter = current[f"Fighter {j}"]

            stats = pd.concat([df.iloc[i+1:,].loc[df[f"Fighter {k}"] == current_fighter] for k in range(1,3)])

            if stats.empty:
                current_winrate = 0

            else:
                wins = 0 if stats.loc[stats["Winner"] == current_fighter].empty else (stats["Winner"].value_counts())[current_fighter]
                all_fights = len(stats.index)

                current_winrate = 1 if not (wins - all_fights) else round(wins/all_fights, 2)
            
            winrate += [current_winrate]

    winrate = np.array(winrate)
    winrate2d = winrate.reshape(nrows, 2)

    winrate_df = pd.DataFrame({"Current winrate F1": winrate2d[:, 0], "Current winrate F2": winrate2d[:, 1]})

    res = pd.concat([df, winrate_df], axis=1, ignore_index=True)
    res.columns = df.columns.to_list() + winrate_df.columns.to_list()
    
    return res.drop(["index"], axis=1)


winrates_df = history_winrate(df_man_fixed)



In [98]:
names = [ele.replace(" ", "_") for ele in winrates_df.columns.to_list()]

winrates_df.columns = names

In [99]:
# model_Winner=smf.logit("")

model_Time=smf.ols(f"Time_in_seconds ~ {' + '.join(winrates_df.drop(['Location', 'Fighter_1', 'Fighter_2', 'Time', 'Winner', 'Weight_Class', 'Method', 'Round', 'Time_in_seconds', 'Lightweight'], axis=1))}", data=winrates_df).fit()
print(model_Time.summary())

                            OLS Regression Results                            
Dep. Variable:        Time_in_seconds   R-squared:                       0.664
Model:                            OLS   Adj. R-squared:                  0.663
Method:                 Least Squares   F-statistic:                     699.9
Date:                Mon, 08 Apr 2024   Prob (F-statistic):               0.00
Time:                        10:46:08   Log-Likelihood:                -43335.
No. Observations:                6405   AIC:                         8.671e+04
Df Residuals:                    6386   BIC:                         8.684e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept            480.0704      9

In [109]:
# fighter1= winner -> winner is a column that shows whether figher 1 has won

winrates_df["Winner"] = np.where(winrates_df["Fighter_1"] == winrates_df["Winner"], 1, 0)

model_logit=smf.logit(f"Winner ~ {' + '.join(winrates_df.drop(['Location', 'Fighter_1', 'Fighter_2', 'Time', 'Winner', 'Weight_Class', 'Method', 'Round', 'Time_in_seconds', 'Lightweight'], axis=1))}", data=winrates_df).fit()
print(model_logit.summary())

# winrates_df.drop(['Location', 'Fighter_1', 'Fighter_2', 'Time', 'Winner', 'Weight_Class', 'Method', 'Round', 'Time_in_seconds', 'Lightweight'], axis=1)


         Current function value: 0.000000
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                 Winner   No. Observations:                 6405
Model:                          Logit   Df Residuals:                     6386
Method:                           MLE   Df Model:                           18
Date:                Mon, 08 Apr 2024   Pseudo R-squ.:                     inf
Time:                        10:50:54   Log-Likelihood:            -1.6017e-06
converged:                      False   LL-Null:                        0.0000
Covariance Type:            nonrobust   LLR p-value:                     1.000
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept            -20.3425   2824.257     -0.007      0.994   -5555.784    5515.099
Fighter_1_KD          -2.9349   4196.143 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect